In [36]:
import numpy as np
import pandas as pd
import os
import re
import copy
#from snowballstemmer import stemmer

def urdu_stemming(word):
  word2 = copy.deepcopy(word)
  suffixes = ["یں", "وں", "ے", "یاں"]
  for suffix in suffixes:
    if word2.endswith(suffix):
      word2 = word2[:-len(suffix)]
  return word

def remove_non_urdu(text):
    urdu_alphabets = u'\u0600-\u06FF'
    pattern = f'[^{urdu_alphabets}]'
    text = re.sub(pattern, ' ', text)
    text = re.sub('[,.;:]', '', text)
    return text

stop_words = set(pd.read_csv("Urdu stopwords.txt", squeeze=True).values)

global file_count
file_count = 0

global term_id
term_id = 0

global term_id1
term_id1 = {}

global forward_index
forward_index = {}

def process_dir(dir_name):
    global file_count
    global term_id

    for file in os.listdir(dir_name):
        file_count += 1
        file_name = f"{file_count}/{file}\n"
        with open("docids.txt", "a") as f:
            f.write(file_name)

        df = pd.read_csv(dir_name + file , error_bad_lines=False)
        data = remove_non_urdu(df.to_string())

        tokens = data.split()
        tokens = [urdu_stemming(i) for i in tokens if i not in stop_words]

        for word in tokens:
            if word not in term_id1:
                term_id1[word] = term_id
                term_id += 1 

        with open(os.path.join(dir_name, file), 'r') as file1:
            text = file1.read() 

        pos_dict = {term_id1[word]:[i for i, x in enumerate(tokens) if x == word] for word in text.split() if word in term_id1}

        tokens1=text.split()
        
        indices=[]
        for word in set(tokens1):
                indices = [i for i in range(len(tokens1)) if tokens1 [i] == word]
                #indices = [i+1 for i in range(len(tokens1)) if tokens1[i] == word]
                # for i in range(len(tokens1)):
                #     if tokens1[i] == word:
                #         print("meow : ",tokens1[i],word)
                if word in term_id1:
                    pos_dict[term_id1[word]]=indices
        forward_index[file] = pos_dict
        
        #store the forward index in a txt file in append mode
        with open("doc_index.txt", "a") as f:
            #first write the file name SPACE term_id SPACE position
            for key, value in forward_index[file].items():
                f.write(str(file_count) + " " + str(key) + " ")
                for i in value:
                    f.write(str(i) + " ")
                f.write("\n")
            
     
    for key, value in term_id1.items():
        #store key and value in a txt file in append mode
        with open("termids.txt", "a") as f:
            f.write(str(key) + "/" + str(value) + "\n")

path="C:/Users/HP/Downloads/Documents (2)/Documents/"

process_dir(path)

b'Skipping line 5: expected 1 fields, saw 2\n'
b'Skipping line 5: expected 1 fields, saw 2\n'
b'Skipping line 5: expected 1 fields, saw 2\n'
b'Skipping line 5: expected 1 fields, saw 2\n'
b'Skipping line 5: expected 1 fields, saw 2\n'
b'Skipping line 5: expected 1 fields, saw 2\n'
b'Skipping line 5: expected 1 fields, saw 21\n'
b'Skipping line 5: expected 1 fields, saw 2\n'
b'Skipping line 5: expected 1 fields, saw 2\n'
b'Skipping line 5: expected 1 fields, saw 2\n'
b'Skipping line 5: expected 1 fields, saw 2\n'
b'Skipping line 5: expected 1 fields, saw 2\n'
b'Skipping line 5: expected 1 fields, saw 24\n'
b'Skipping line 5: expected 1 fields, saw 19\n'
b'Skipping line 5: expected 1 fields, saw 2\n'
b'Skipping line 5: expected 1 fields, saw 3\n'
b'Skipping line 5: expected 1 fields, saw 2\n'
b'Skipping line 5: expected 1 fields, saw 2\n'
b'Skipping line 5: expected 1 fields, saw 3\n'
b'Skipping line 5: expected 1 fields, saw 2\n'
b'Skipping line 5: expected 1 fields, saw 2\n'
b'Skipping

In [11]:
global inverted_index
inverted_index = {}

#read the forward_index.txt file and store it in a dictionary
with open("doc_index.txt", "r") as f:
    for line in f:
        line = line.split()
        file = line[0]
        term_id = int(line[1])
        positions = [int(x) for x in line[2:]]
        if term_id not in inverted_index:
            inverted_index[term_id] = {}
        inverted_index[term_id][file] = positions


previous_key=0
#store the inverted index in a txt file in append mode
with open("term_index.txt", "a") as f:
    #first store term_id
    for key, value in inverted_index.items():
        f.write(str(key) + " ")
        #then store the file name and the position
        previous_key=0
        for key1, value1 in value.items():
            f.write(str(int(key1)-int(previous_key)) + ":")
            previous_key=key1
            previous_i=0
            for i in value1:
                f.write(str(i-previous_i) + " ")
                previous_i=i

        f.write("\n")


doc1=0
temp={}
doc1={}
pos1={}
#read the inverted_index.txt file and store the number of occurence of each term in a dictionary temp and number of doucments in a variable
with open("term_index.txt", "r") as f:
    length=0

    for line in f:
        a = line.split()
        #if there is a : in the line then increment the number of documents
        term_id = int(a[0])
        pos1[term_id]=length


        temp[term_id]=len(a)-1
        
        b=line.split(":")
        doc1[term_id]=len(b)-1 

        length=len(line)+length
        

#store the number of occurence of each term in a txt file in append mode
with open("term_info.txt", "a") as f:
    for key, value in temp.items():
        f.write(str(key) + "/" + str(pos1[key]) +"/" + str(value)+"/"+ str(doc1[key]) + "\n")

NameError: name 'doc_info' is not defined